# 线性回归的简洁实现

In [ ]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l
true_w = torch.tensor([2, -3.4])
true_b = 4.2
features, labels = d2l.synthetic_data(true_w, true_b, 1000) #设定标准答案以及根据标准答案来出题产生1000个数据


## 理解features 和labels 【非常重要】
 - features（特征）：它就是一个 Excel 表格。里面有 1000 行、2 列。你可以把它当成 1000 个机器人的 “身高”和“体宽”。

 - labels（标签）：它也是一个 Excel 表格。里面有 1000 行、1 列。这是这 1000 个机器人对应的 “真实体重”（标准答案）。

 注意！！因为在我们的任务里，身高和体宽是“已知条件”（线索），而体重是我们要猜的“未知结果”（目标）。你想象一下你在做一道数学应用题：题目（已知线索）：小明买了 2 斤苹果，每斤 5 块钱。（这是 2 个特征参数，相当于 2 列）。问题（未知结果）：请问小明一共花了多少钱？
答案（标签）：10 块钱。（这是 1 个最终结果，相当于 1 列）。你看，2 个线索，推导出 1 个答案，这完全没毛病对吧？ 你不需要让 AI 去猜“苹果几斤、几块钱”，因为这些是你喂给它的前提条件！在你的机器人例子里，你用尺子量出了它的身高和体宽（2个特征），你想让 AI 帮你算出它的重量（1个标签）。多因一果，这就是为什么特征有 2 列，而标签只有 1 列

 ## 既然列数不一样，它们到底是怎么“对应”上的？
PyTorch 的“订书机”机制（TensorDataset），根本不看你有几列，它只看你有几行！

它是按“行（第几个机器人）”来一一对应的：

第 1 行对应：

左手边（特征箱子抽出第 1 行）：[180厘米, 50厘米]  👉 这是 1号机器人 的线索。

右手边（标签箱子抽出第 1 行）：[100公斤] 👉 这是 1号机器人 的真实重量答案。

咔嚓！钉在一起！ AI 拿到这张卡片就懂了：“哦，原来 180 高、50 宽的家伙，重 100 公斤啊！”

第 2 行对应：

左手边（特征箱子抽出第 2 行）：[160厘米, 40厘米] 👉 这是 2号机器人 的线索。

右手边（标签箱子抽出第 2 行）：[70公斤] 👉 这是 2号机器人 的真实重量答案。

咔嚓！钉在一起！

发现了吗？特征的列数可以无限多！
哪怕你以后收集了机器人的 身高、体宽、材质、电池容量、电机型号（5 列特征），但你要预测的依然只是它的重量（1 列标签）。订书机依然是把“1号机器人的 5 个特征”和“1号机器人的 1 个重量”钉在一起。

## 读取数据集 [非常重要]

In [ ]:
def load_array(data_arrays,batch_size,is_train=True):
    dataset = data.TensorDataset(*data_arrays)#接受任意多个参数并将它放在一个元组里面
    return data.DataLoader(dataset,batch_size,shuffle=is_train)
#设定一下每次拿多少个
batch_size = 10
data_iter = load_array((features,labels),batch_size)


自然的逻辑：在现实中，零件（features）和重量说明书（labels）是分开的两个大箱子。如果直接丢进机器，第 1 个零件可能会配上第 500 个说明书，彻底乱套！

TensorDataset 在干嘛：它就像一个订书机。它会把第 1 个零件和第 1 个说明书死死钉在一起，做成一张“卡片”。钉完之后，我们就得到了 1000 张绝对不会错乱的“带答案的识字卡”。

# 神经网络的预参数设定

In [3]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

如果我现在让你处理一个新任务——“收集了机器人的 5 个传感器数据（5个特征），要预测它的 3 个关节角度（3个输出标签）”。你能一秒钟反应过来，代码要改成 
- net = nn.Sequential(nn.Linear(5, 3))


*nn.Sequential(...)：这是一个“流水线传送带”*
虽然现在我们的模型极其简单，只包含 Linear 这一台机器，
 - 外层 nn.Sequential(...)——流水线/传送带

Sequential 的作用是把多个层串成一条线

但是由于以后几乎所有的深度学习模型都是多层的，在这里使用 Sequential 会让你提前熟悉“标准的流水线”操作。以后你要加机器，只需要往里面用逗号隔开一直塞就行了
- nn.Sequential(机器A, 机器B, 机器C)。

 ## 初始化模型参数 [看懂即可]

In [5]:
net[0].weight.data.normal_(0, 0.01)
net[0].bias.data.fill_(0)

tensor([0.])

既然 nn.Linear 已经在肚子里偷偷建好了旋钮 $w$（权重）和 $b$（偏置），那在我们通电开机之前，总得给它们一个初始的位置吧？net[0]：就是指代我们流水线上的第 1 台机器（也就是那个 Linear 层）。编程里计数是从 0 开始的。weight.data.normal_(0, 0.01)：我们通过 .weight.data 访问到这台机器的权重旋钮，然后用替换方法 normal_ 把它的初始值设定为一个从均值为 0、标准差为 0.01 的正态分布中随机采样出来的小数。bias.data.fill_(0)：我们通过 .bias.data 访问偏置旋钮，用替换方法 fill_ 把偏置参数初始化为零。

In [6]:
#计算误差用的是MSELoss
loss = nn.MSELoss()
trainer = torch.optim.SGD(net.parameters(),lr=0.03)

NameError: name 'torch' is not defined

人话翻译：MSE 是 Mean Squared Error（均方误差）的缩写。这就相当于你在车间里挂了一个“全自动计分板”。它干了什么？ 它完美替代了上一节课里那个丑陋的 (y_hat - y.reshape(y_hat.shape))**2 / 2。默认情况下，它会自动把所有样本的损失加起来，然后算出一个平均值返回给你。计算均方误差使用的就是这个 MSELoss 类，数学上也叫它平方$L_2$范数。掌握到什么水平？ 死死记住！ 以后只要你是做“预测具体数值”的任务（比如预测机器人的重量、预测电机的温度），闭着眼睛写这行代码就对了。

-------------------------------------------------- 

net.parameters()：这是你把模型（net）里所有的旋钮（权重 $w$ 和偏置 $b$）一把抓过来，全部交给了这位纠错大师。大师拿到了控制权，一会儿才知道该去拧谁。lr=0.03：lr 是 Learning Rate（学习率）。这是你给大师定的规矩：每次拧旋钮的时候，步子（幅度）迈多大。小批量随机梯度下降只需要设置这一个 lr 值即可。它替代了什么？ 它完美替代了上一节课你大呼熟悉的那个鱼书核心公式：param -= lr * param.grad / batch_size。PyTorch 在 optim 模块中实现了这种算法的标准工具和许多变种。

# 一定要背下来的用于【训练的代码】

In [7]:
# 深度学习标准训练流程 

import torch
from torch import nn
from torch.utils import data

# ==========================================
# 第一步：准备“发牌机” (处理数据)
# ==========================================
# 假设 features 是特征，labels 是答案
# 1. TensorDataset: 把特征和答案像订书机一样一一对应钉起来
dataset = data.TensorDataset(features, labels)

# 2. DataLoader: 造一个发牌机，每次随机发 10 张牌 (batch_size=10)，并且洗牌 (shuffle=True)
data_iter = data.DataLoader(dataset, batch_size=10, shuffle=True)


# ==========================================
# 第二步：准备“车间三大件” (模型、计分板、纠错大师)
# ==========================================
# 1. 模型 (Model)：造一个流水线，里面放一台线性全连接机器 (输入2个特征，输出1个预测值)
net = nn.Sequential(nn.Linear(2, 1))

# 2. 损失函数 (Loss)：挂一个均方误差计分板
loss = nn.MSELoss()

# 3. 优化器 (Optimizer)：请一位随机梯度下降(SGD)纠错大师，接管模型所有参数，规定步长 lr=0.03
trainer = torch.optim.SGD(net.parameters(), lr=0.03)


# ==========================================
# 第三步：训练大循环 (⭐⭐⭐ 核心死记硬背区)
# ==========================================
num_epochs = 3 # 总共练 3 遍

for epoch in range(num_epochs):
    for X, y in data_iter:              # 发牌机发牌 (拿出一小批 X 和 y)
                                        # X：10×2 的张量（10个样本，每个2个特征）
                                        # y：10×1 的张量（10个标准答案）
        
        l = loss(net(X), y)             # 动作1：前向传播 ( net(X)，并用计分板算分 l)
                                        #等价于net（X）再loss（……，y）
        trainer.zero_grad()             # 动作2：清空计算器 (把大师脑子里上次的结果清零)
        
        l.backward()                    # 动作3：反向传播 (追责！自动算出每个参数该背多少锅 )
                             
        trainer.step()                  # 动作4：更新参数对应 #3优化器 (Optimizer)  (大师根据锅的大小，动手调整模型的旋钮)

SyntaxError: invalid character '∂' (U+2202) (3621176430.py, line 46)

##　⭐⭐⭐⭐一句话记忆：在 PyTorch 里，模型对象加括号 = 执行前向传播　－－－》动作１　

- 先搞清楚"前向传播"这四个字
- "前向"就是从左往右、从输入到输出，数据像水流一样单向流过整台机器。
  
### `net(X)` 计算过程
`net(X)` 的计算仅包含以下步骤：

#### 1. 计算公式
$$
\hat{y} = x_1 \times w_1 + x_2 \times w_2 + b
$$

#### 2. 代入数值
$$
\hat{y} = 0.5 \times 0.0032 + (-1.0) \times (-0.0015) + 0
$$

#### 3. 计算结果
$$
\hat{y} = 0.0016 + 0.0015 + 0 = 0.0031
$$


输入 X ──→ 【机器内部计算】──→ 输出 ŷ（预测值）
而且当有10个样本，也就是一个 batch 的时候，net x 会同时算10个预测值，也就是矩阵的运算
---

# PyTorch 自动求导

PyTorch 会自动计算损失函数 $l$ 对参数的偏导数：

- 对权重 $w_1$ 的偏导数：
$$
\frac{\partial l}{\partial w_1}
$$

- 对权重 $w_2$ 的偏导数：
$$
\frac{\partial l}{\partial w_2}
$$

- 对偏置 $b$ 的偏导数：
$$
\frac{\partial l}{\partial b}
$$

计算出的梯度会自动保存在对应参数的 `.grad` 属性中。

-----------

loss(net(X), y) 完整链条
现在把前向传播和损失函数串起来：

l = loss(net(X), y)

Python 的执行顺序是从内到外，所以拆成两步：

第①步：先算 net(X)     → 得到 ŷ（10个预测值）
第②步：再算 loss(ŷ, y) → 得到 l（一个损失数字）

# ⭐⭐⭐⭐四行核心训练代码详解
## 第1行：为什么要赋值给 l？
```python
l = loss(net(X), y)
```
先想一个问题：如果不存下来会怎样？
```python
loss(net(X), y)         # 算完就扔了，没人记住这个结果
```
下一行你需要对它做 `.backward()`，但你找不到它了，因为你没给它起名字！

### 类比
```python
# 你去超市买东西
价格 = 收银台.扫码(苹果)    # 把价格存到变量里
print(价格)                 # 后面还能用

# 如果你不存：
收银台.扫码(苹果)           # 扫完了，价格显示了一下就消失了
print(???)                  # 你想查价格？没了，找不到了
```
所以 `l` 就是一个名字，用来"记住"这次算出来的损失值，方便后面用它做反向传播。

`l` 是 loss 的首字母缩写，你叫它 abc 也行，只是习惯用 l。

---

## 第3行（先讲这行）：为什么用 l 做反向传播？
```python
l.backward()
```
`l` 不只是一个数字！
你以为 `l` 就是一个普通的数字比如 0.35？不是！

`l` 是一个带着"记忆"的特殊张量。它记住了自己是怎么算出来的：

#### l 的记忆链条：
```
l ← 是由 loss 函数算出来的
  ← loss 的输入是 net(X) 和 y
    ← net(X) 是用 w 和 b 算出来的
      ← w 和 b 是模型的参数
```
PyTorch 在前向传播的时候，偷偷帮你画了一张图：
```
w ──┐
    ├──→ net(X) ──→ ŷ ──┐
b ──┘                    ├──→ loss ──→ l
                    y ───┘
```
当你调用 `l.backward()` 时，PyTorch 就沿着这张图，从右往左倒着走：
```
l ──→ 算出 ∂l/∂ŷ ──→ 算出 ∂l/∂w 和 ∂l/∂b
```
它算出的就是：

"如果我把 $w_1$ 调大一点点，$l$ 会变大还是变小？"
"如果我把 $w_2$ 调大一点点，$l$ 会变大还是变小？"
"如果我把 $b$ 调大一点点，$l$ 会变大还是变小？"

这些答案（梯度）被存在：
```python
net[0].weight.grad   # w的梯度，存在这里
net[0].bias.grad     # b的梯度，存在这里
```

---

## 第2行：zero_grad() 为什么要清空梯度？
```python
trainer.zero_grad()
```
### 先看一个灾难场景
假设不清零：
```
第1批数据：backward() → w的梯度 = 0.5
第2批数据：backward() → w的梯度 = 0.5 + 0.3 = 0.8  ← 累加了！
第3批数据：backward() → w的梯度 = 0.8 + 0.2 = 1.0  ← 又累加了！
```
PyTorch 的默认行为是：新梯度会叠加到旧梯度上面！

这是 PyTorch 故意设计的（某些高级场景需要累加），但对我们来说是灾难——我们每批数据应该独立算梯度。

### 加上清零之后
```
第1批数据：zero_grad() → 梯度=0 → backward() → 梯度=0.5  ✓ 干净的
第2批数据：zero_grad() → 梯度=0 → backward() → 梯度=0.3  ✓ 干净的
第3批数据：zero_grad() → 梯度=0 → backward() → 梯度=0.2  ✓ 干净的
```

### 类比
```
你用计算器：
  算第1题：3+5=8     ← 对的
  算第2题：没按清零，直接按 +2×3=14   ← 错了！8还留着呢！

正确做法：
  算第1题：3+5=8
  按 AC（清零）
  算第2题：2×3=6     ← 对了
```
`trainer.zero_grad()` 就是按计算器的 AC 键。

---

## 第4行：trainer.step() 从哪来的？干什么的？
```python
trainer.step()
```
### 它从哪来？
往上翻，你定义过这行：
```python
trainer = torch.optim.SGD(net.parameters(), lr=0.03)
```
`trainer` 就是你在这行创建的优化器对象，它是 SGD（随机梯度下降）。

就像 `net` 可以加括号调用一样，`trainer` 也有自己的方法，其中最重要的就是 `.step()`。

### 它干什么？
`.step() = 真正动手改参数！`

前面所有步骤都是"分析"，只有 step() 是"动手"：
```python
# step() 内部做的事情（简化版）：
w = w - lr × w的梯度
b = b - lr × b的梯度
```

### 用具体数字：
```
假设当前：
  w₁ = 0.5
  w₁的梯度 = 2.0
  lr = 0.03

执行 step() 后：
  w₁ = 0.5 - 0.03 × 2.0 = 0.5 - 0.06 = 0.44
```
参数被"拧"了一下，朝着让损失变小的方向调整了。

### 类比
```
你考试考了60分（l = 损失很大）

l.backward()  → 分析原因：数学太差（梯度告诉你方向）
trainer.step() → 真正行动：回家刷数学题（更新参数）

下次考试考了70分（l 变小了）
```
`backward()` 是军师出谋划策，`step()` 是将军带兵冲锋。

---

## 四行代码的完整故事
```python
# 场景：发牌机给了10道题（X和y）

l = loss(net(X), y)     # ① 做题+批改，得到分数l
trainer.zero_grad()      # ② 把上次的草稿纸擦干净 ！！注意trainer写的是所有的parameters因此都清理干净的
l.backward()             # ③ 分析"哪里做错了，该怎么改"
trainer.step()           # ④ 真正改正：调整 w 和 b
```

### 时间线：
```
─────────────────────────────────────────────→

  ①做题得分   ②擦草稿纸   ③分析错因   ④改正参数
  l=0.35      梯度清零     算梯度      w和b更新
     │                       │            │
     │    前向传播的产物       │  反向传播    │ 参数更新
     └───────────────────────┴────────────┘
              一个batch的完整处理流程
```

### 为什么顺序是 ① → ② → ③ → ④？
| 能不能换顺序？ | 结果 |
|----------------|------|
| ② 和 ① 换？zero_grad() 放在 loss 前面？ | 可以！ 很多人确实这么写，效果一样 |
| ③ 放在 ② 前面？先 backward() 再 zero_grad()？ | 不行！ 你刚算出的梯度会被清掉 |
| ④ 放在 ③ 前面？先 step() 再 backward()？ | 不行！ 还没算梯度你拿什么更新？ |

唯一的硬性要求：② 清零必须在 ③ 算梯度之前，③ 算梯度必须在 ④ 更新之前。

问题一：偏置 $b$ 到底是从哪冒出来的？！你非常敏锐！你发现 nn.Linear(2, 1) 里面明明只写了 2（输入特征）和 1（输出结果），这说明它造了 2 个权重 $w$，那偏置 $b$ 呢？我根本没写啊！答案是：PyTorch 偷偷白送给你了！在 PyTorch 的底层源码里，nn.Linear 这个机器的完整说明书其实长这样：nn.Linear(in_features, out_features, bias=True)看到那个 bias=True 了吗？这是框架的出厂默认设置！框架的创造者知道，全天下 99.99% 的线性模型和神经网络，都必须带偏置 $b$。所以为了让你少打几个字，只要你写 nn.Linear(2, 1)，它在底层就会自动给你生成一个 $2 \times 1$ 的权重矩阵 $w$，同时再悄悄生成一个偏置标量 $b$。如果哪天你真的遇到了一个极其奇葩的任务，坚决不需要 $b$，你必须手动强制关掉它：写成 nn.Linear(2, 1, bias=False)。总结：不是你没设置，而是 PyTorch 把它当成标配，帮你默认开启了！

问题二：net 到底是个啥？为什么可以像函数一样写成 net(X)？你觉得奇怪，net = nn.Sequential(nn.Linear(2, 1)) 明明是一个赋值语句，定义了一个“流水线”对象，怎么到了下面，它摇身一变，就能加上括号当成函数 net(X) 来用了呢？这是 Python 语言和 PyTorch 结合产生的一种**“魔法机制”**：net 是一台物理存在的机器：当你写下 net = nn.Sequential(...) 的时候，你相当于在车间里把这台名叫 net 的机器实体给造出来了。这台机器肚子里装着刚才说的 $w$ 和 $b$。net(X) 相当于按下机器的“启动按钮”：在 PyTorch 里，所有的神经网络模型（也就是 nn.Module 的子类），都被赋予了一个隐藏技能。只要你在机器名字后面加上括号，并且往里面塞入材料 X，机器就会自动启动内部的齿轮，执行前向传播（Forward）。底层到底干了啥？ 当你执行 net(X) 时，PyTorch 在底层其实是在帮你执行：X 乘以 肚子里的 w + 肚子里的 b，然后把算出来的预测值（那个 1 个特征的输出结果）吐给你。大白话：net 是电饭煲，X 是生米。net(X) 的意思就是**“把生米倒进电饭煲，盖上盖子通电，等它煮成熟饭（预测结果）吐出来”**。

问题三：最核心的死结！为什么是 loss(net(X), y)，而不是 loss(net(features), labels)？
这个问题是你走向“工程实战”必须要迈过去的一道超级大坎！我们回到那个“车间主任与发牌机”的故事：

features 和 labels 是什么？ 它们是你车间仓库里存放的 1000 个零件的总集。

发牌机 data_iter 在干嘛？ 你的内存条根本吃不下一次性把 1000 个零件全塞进机器里算。所以发牌机 for X, y in data_iter: 的作用是，每次只去仓库里搬 10 个零件出来！

这里的 X，就是发牌机刚刚发给你的 10 个零件的尺寸。

这里的 y，就是这 10 个零件对应的标准重量。

🚨 灾难预演：如果你写成了 loss(net(features), labels) 会发生什么？

假如你真的这么写了，你的循环就变成了这样：

发牌机给你发了 10 张牌（X 和 y）。

你把这 10 张牌扔到一边看都不看。

你跑到仓库里，把所有的 1000 个 features 强行塞进 net 机器里预测。

你拿这 1000 个预测结果，和 1000 个 labels 去算 Loss 分数